In [1]:
# 라이브러리
import seaborn as sns 
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
import time
import joblib
import json
from datetime import datetime
# 모델 라이브러리
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor


# 데이터 가져오기
# 훈련 데이터
train = pd.read_csv("train.csv")
train.head(1)
# 테스트 데이터
test = pd.read_csv("test.csv")
test.head(1)

# 수치형 / 범주형 컬럼 나누기
# numeric_features = train.select_dtypes(include=['int64','float64'].columns.tolist()
numeric_features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
category_features = ['Sex']

# 독립변수, Target 설정
X = train[numeric_features + category_features]
X
y = train['Calories']
y


# 학습 / 검증 데이터 분할 함수
num_bins = 20
y_binned = pd.cut(y, bins=num_bins, labels=False)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y_binned)

# Target 데이터 로그변환 : RMSLE 평가지표를 따라가기 위함
y_train = np.log1p(y_train)
y_train
y_val = np.log1p(y_val)
y_val

# # 데이터 가공
# # one-hot encoding, scaler
# encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# X_train_category = encoder.fit_transform(X_train[category_features])
# X_val_category = encoder.transform(X_val[category_features])

# scaler = StandardScaler()
# X_train_numeric = scaler.fit_transform(X_train[numeric_features])
# X_val_numeric = scaler.transform(X_val[numeric_features])

# # 가공한 데이터 합치기 (numeric+category)
# X_train_combined = pd.concat([
#         pd.DataFrame(X_train_category, columns=['sex1','sex2']),
#         pd.DataFrame(X_train_numeric, columns=numeric_features)
#     ], axis=1)
# X_val_combined = pd.concat([
#         pd.DataFrame(X_val_category, columns=['sex1','sex2']),
#         pd.DataFrame(X_val_numeric, columns=numeric_features)
#     ], axis=1)


# # 모델 생성 / 학습 / 성능비교
# models = {
#     'XGBoost' : XGBRegressor(random_state=42, n_jobs=-1),
#     'RandomForest' : RandomForestRegressor(random_state=42, n_jobs=-1),
#     'LightGBM' : LGBMRegressor(random_state=42, n_jobs=-1),
#     'CatBoost' : CatBoostRegressor(random_seed=42,
#                     verbose=100,
#                     early_stopping_rounds=50,
#                     loss_function='RMSE'
#                 )
# }

# # 각 모델별 하이퍼파라미터 후보군 정의
# param_grid = {
#     'XGBoost': [
#         {'n_estimators':100, 'learning_rate': 0.05, 'max_depth': 4},
#         {'n_estimators':200, 'learning_rate': 0.01, 'max_depth': 6}
#     ],
#     'RandomForest': [
#         {'n_estimators': 100, 'max_depth': 5},
#         {'n_estimators': 200, 'max_depth': 10}
#     ],
#     'LightGBM': [
#         {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': -1},
#         {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 6}
#     ],
#     'CatBoost': [
#         {'iterations': 500, 'learning_rate': 0.05, 'depth': 6, 'l2_leaf_reg': 3, 'subsample': 0.8, 'random_strength': 5},
#         {'iterations': 1000, 'learning_rate': 0.1, 'depth': 8, 'l2_leaf_reg': 10, 'subsample': 1.0, 'random_strength': 10}
#     ]
# }

# # 교차 검증을 통한 최적 모델 선택
# best_score = 0 # 최고 성능 점수
# best_model_name = None
# best_model = None


# # 각 모델과 하이퍼 파라미터 조합에 대해 교차검증 수행
# for model_name, model in models.items():
#     print(f"\n--- Testing {model_name} ---")
#     start_time = time.time()
#     for params in param_grid[model_name]:
#         model.set_params(**params)
#         cv_scores = cross_val_score(model, X_train_combined, y_train, cv=5, scoring='r2')
#         mean_cv = np.mean(cv_scores)  # 평균 교차검증 점수
#         end_time = time.time()
#         elapsed_time = end_time - start_time
#         print(f"Params: {params}, CV R2 Score: {mean_cv:.4f}, Training Time: {elapsed_time:.2f} seconds")
        
#         # 최고 성능 모델 업데이트
#         if mean_cv > best_score:
#             best_score = mean_cv
#             best_model_name = model_name
#             best_model = model.set_params(**params)


# # 최종 선택된 모델로 테스트셋 평가
# best_model.fit(X_train_combined, y_train)  # 최적 모델 학습
# y_pred = best_model.predict(X_val_combined)  # 테스트셋 예측
# test_r2 = r2_score(y_val, y_pred)  # 테스트셋 정확도 계산


# # 최고 성능 모델 저장
# model_info = {
#     'model':best_model,
#     'scaler':scaler,
#     'encoder':encoder
# }
# joblib.dump(model_info, 'model.pkl')
# # with open('model_info.json', 'w') as f:
# #     json.dump(model_info, f)

# # 최종 결과 출력
# print(f"\nBest Model: {best_model_name}")
# print(f"Best CV Score: {best_score:.4f}")
# print(f"Test Set R2: {test_r2:.4f}")
# print("\nModel has been saved.")

532263    3.135494
580340    2.484907
101839    5.111988
438420    4.762174
449976    4.094345
            ...   
269955    4.143135
502294    4.997212
58461     3.367296
647977    5.087596
245703    2.890372
Name: Calories, Length: 150000, dtype: float64

In [3]:
# 필요한 라이브러리 임포트
import pandas as pd  # 데이터 처리를 위한 pandas
import numpy as np  # 수치 계산을 위한 numpy
from sklearn.model_selection import train_test_split, cross_val_score  # 데이터 분할 및 교차 검증
from sklearn.impute import SimpleImputer  # 결측치 처리
from sklearn.preprocessing import OneHotEncoder, StandardScaler  # 범주형 변수 인코딩 및 수치형 변수 스케일링
from sklearn.compose import ColumnTransformer  # 컬럼별 전처리 파이프라인 구성
from sklearn.pipeline import Pipeline  # 전체 전처리 및 모델링 파이프라인 구성
from sklearn.linear_model import Ridge, Lasso, ElasticNet  # 선형 회귀 모델
from sklearn.metrics import mean_squared_error, make_scorer  # 모델 평가 지표
from sklearn.model_selection import RandomizedSearchCV  # 랜덤 서치를 통한 하이퍼파라미터 튜닝
from skopt import BayesSearchCV  # 베이지안 최적화를 통한 하이퍼파라미터 튜닝
import xgboost as xgb  # XGBoost 모델
import lightgbm as lgb  # LightGBM 모델
import warnings  # 경고 메시지 처리
import joblib  # 모델 저장 및 로드
import os  # 파일 시스템 작업
import json  # JSON 파일 처리
# LightGBM의 불필요한 경고 메시지 무시 설정
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')

In [4]:
# numeric_features / category_features

numeric_transformer = Pipeline([
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

preprocessor= ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, category_features)
])

In [5]:
# 모델 생성 / 학습 / 성능비교
models = {
    'XGBoost' : XGBRegressor(random_state=42, n_jobs=-1),
    # 'RandomForest' : RandomForestRegressor(random_state=42, n_jobs=-1),
    'LightGBM' : LGBMRegressor(random_state=42, n_jobs=-1),
    'CatBoost' : CatBoostRegressor(random_seed=42,
                    verbose=100,
                    early_stopping_rounds=50,
                    loss_function='RMSE'
                )
}

In [15]:
param_spaces = {
    'XGBoost': {'reg__n_estimators':[100, 200], 'reg__learning_rate': [0.05, 0.1], 'reg__max_depth': [4, 6]},
    # 'RandomForest': {'reg__n_estimators': [100,200], 'reg__max_depth': [5 ,10]},
    'LightGBM': {'reg__n_estimators': [100,200], 'reg__learning_rate': [0.05, 0.1], 'reg__max_depth': [-1,6]},
    'CatBoost': {'reg__iterations': [500,1000], 'reg__learning_rate': [0.05, 0.1], 'reg__depth': [6,8], 'reg__l2_leaf_reg': [3,10], 'reg__subsample': [0.8,1.0], 'reg__random_strength': [5,10]}
}

In [7]:
# 하이퍼파라미터 적용 전

results = {}  # 결과 저장 딕셔너리

for name, model in models.items():
    print(f'\n==== {name} ====')
    try:
        # 3.1 전체 파이프라인 구성 (전처리 + 모델)
        pipe = Pipeline([
            ('preprocessor', preprocessor),  # 전처리 단계
            ('reg', model)  # 모델 단계
        ])
        
        # 3.2 모델 학습
        pipe.fit(X_train, y_train)
        
        # 3.3 예측 및 성능 평가
        y_pred = pipe.predict(X_val)  # 예측값 생성
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))  # RMSE 계산
        print(f'RMSE: {rmse:.4f}')
        
        results[name] = rmse  # 결과 저장

        
    except Exception as e:
        print(f"Error: {str(e)}")

# 4. 최종 결과 비교 및 정렬
print("\n==== Final Results ====")
results_df = pd.DataFrame(results.items(), columns=['Model', 'RMSE'])  # 결과 데이터프레임 생성
results_df = results_df.sort_values('RMSE')  # RMSE 기준 오름차순 정렬
print(results_df)


==== XGBoost ====
RMSE: 0.0624

==== RandomForest ====
RMSE: 0.0639

==== LightGBM ====
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003523 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 360
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 8
[LightGBM] [Info] Start training from score 4.141196


C:\Users\Admin\Desktop\Predict_Calorie_Project\project\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


RMSE: 0.0625

==== CatBoost ====
Learning rate set to 0.112494
0:	learn: 0.8622609	total: 163ms	remaining: 2m 42s
100:	learn: 0.0612759	total: 3.77s	remaining: 33.6s
200:	learn: 0.0592647	total: 7.49s	remaining: 29.8s
300:	learn: 0.0584388	total: 11s	remaining: 25.6s
400:	learn: 0.0579209	total: 14.6s	remaining: 21.8s
500:	learn: 0.0575332	total: 18.3s	remaining: 18.2s
600:	learn: 0.0572159	total: 21.8s	remaining: 14.5s
700:	learn: 0.0569445	total: 25.3s	remaining: 10.8s
800:	learn: 0.0567128	total: 28.8s	remaining: 7.14s
900:	learn: 0.0565374	total: 32.2s	remaining: 3.53s
999:	learn: 0.0563609	total: 35.6s	remaining: 0us
RMSE: 0.0596

==== Final Results ====
          Model      RMSE
3      CatBoost  0.059594
0       XGBoost  0.062413
2      LightGBM  0.062539
1  RandomForest  0.063941


In [ ]:
# 하이퍼파라미터 적용!!!!

results = {}  # 모델별 RMSE 결과 저장
best_models = {}  # 최적의 모델 저장

for name, model in models.items():
    print(f'\n==== {name} ====')
    try:
        # 3.1 전체 파이프라인 구성 (전처리 + 모델)
        pipe = Pipeline([
            ('preprocessor', preprocessor),  # 전처리 단계
            ('reg', model)  # 모델 단계
        ])

        # 베이지안 서치
        search = BayesSearchCV(
        # 3.2 랜덤 서치를 통한 하이퍼파라미터 최적화
        # search = RandomizedSearchCV(
            pipe,
            param_spaces[name],
            n_iter=10,  # 탐색 횟수
            cv=5,  # 5-fold 교차 검증
            scoring='neg_root_mean_squared_error',  # 평가 지표 (RMSE)
            random_state=42,  # 재현성을 위한 시드
            n_jobs=-1  # 모든 CPU 코어 사용
        )
        
        # 3.3 모델 학습
        search.fit(X_train, y_train)
        
        # 3.4 최적의 모델 저장
        best_models[name] = search.best_estimator_
        
        # 3.5 교차 검증 결과 계산
        cv_scores = -search.cv_results_['mean_test_score']  # RMSE 값 (음수로 저장되어 있어 부호 변환)
        rmse = np.mean(cv_scores)  # 평균 RMSE
        rmse_std = np.std(cv_scores)  # RMSE 표준편차
        
        print(f'Best RMSE: {rmse:.4f} (+/- {rmse_std:.4f})')
        print(f'Best parameters: {search.best_params_}')
    
        results[name] = rmse
        
    except Exception as e:
        print(f"Error: {str(e)}")

# 4. 최종 결과 분석
print("\n==== Final Results ====")
results_df = pd.DataFrame(results.items(), columns=['Model', 'RMSE'])
results_df = results_df.sort_values('RMSE')  # RMSE 기준 오름차순 정렬
print(results_df)

best_model_name = results_df.iloc[0]['Model']
best_model = best_models[best_model_name]

In [11]:
best_model = models['CatBoost']

# test.head()
X_test = test[numeric_features + category_features]

y_pred = pipe.predict(X_test)  # 예측값 생성
y_pred = np.exp(y_pred) 

submission = pd.DataFrame({
    'id' : test['id'],
    'Calories' : y_pred
})
# 현재 날짜와 시간을 파일명에 포함
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
submission.to_csv(f'submission_{current_time}.csv', index=False)


NotFittedError: Pipeline is not fitted yet.

In [14]:
best_model

# test.head()
X_test = test[numeric_features + category_features]

y_pred = best_model.predict(X_test)  # 예측값 생성
y_pred = np.exp(y_pred) 

submission = pd.DataFrame({
    'id' : test['id'],
    'Calories' : y_pred
})
# 현재 날짜와 시간을 파일명에 포함
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
submission.to_csv(f'submission_{current_time}.csv', index=False)

CatBoostError: There is no trained model to use predict(). Use fit() to train model. Then use this method.